<a href="https://colab.research.google.com/github/dsercam/TC033/blob/main/A4_DL_TC5033_text_generator_Team44.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color='darkorange'><b> TC 5033 :: Advanced Machine Learning Methods </b> </font>
### <font color='darkgray'><b> Activity 4: Building a Simple LSTM Text Generator using WikiText-2</b></font></br></br>
###<font color='darkblue'><b>  Group 44 </b></font>
***Dante Rodrigo Serna Camarillo A01182676***</br>
***Carlos Roberto Torres Ferguson A01215432***</br>


## TC 5033
### Text Generation

<br>

#### Activity 4: Building a Simple LSTM Text Generator using WikiText-2
<br>

- Objective:
    - Gain a fundamental understanding of Long Short-Term Memory (LSTM) networks.
    - Develop hands-on experience with sequence data processing and text generation in PyTorch. Given the simplicity of the model, amount of data, and computer resources, the text you generate will not replace ChatGPT, and results must likely will not make a lot of sense. Its only purpose is academic and to understand the text generation using RNNs.
    - Enhance code comprehension and documentation skills by commenting on provided starter code.
    
<br>

- Instructions:
    - Code Understanding: Begin by thoroughly reading and understanding the code. Comment each section/block of the provided code to demonstrate your understanding. For this, you are encouraged to add cells with experiments to improve your understanding

    - Model Overview: The starter code includes an LSTM model setup for sequence data processing. Familiarize yourself with the model architecture and its components. Once you are familiar with the provided model, feel free to change the model to experiment.

    - Training Function: Implement a function to train the LSTM model on the WikiText-2 dataset. This function should feed the training data into the model and perform backpropagation.

    - Text Generation Function: Create a function that accepts starting text (seed text) and a specified total number of words to generate. The function should use the trained model to generate a continuation of the input text.

    - Code Commenting: Ensure that all the provided starter code is well-commented. Explain the purpose and functionality of each section, indicating your understanding.

    - Submission: Submit your Jupyter Notebook with all sections completed and commented. Include a markdown cell with the full names of all contributing team members at the beginning of the notebook.
    
<br>

- Evaluation Criteria:
    - Code Commenting (60%): The clarity, accuracy, and thoroughness of comments explaining the provided code. You are suggested to use markdown cells for your explanations.

    - Training Function Implementation (20%): The correct implementation of the training function, which should effectively train the model.

    - Text Generation Functionality (10%): A working function is provided in comments. You are free to use it as long as you make sure to uderstand it, you may as well improve it as you see fit. The minimum expected is to provide comments for the given function.

    - Conclusions (10%): Provide some final remarks specifying the differences you notice between this model and the one used  for classification tasks. Also comment on changes you made to the model, hyperparameters, and any other information you consider relevant. Also, please provide 3 examples of generated texts.



>> Installed to run in colab

In [1]:
!pip install scikit-plot
!pip install torchdata
!pip install tokenizers
!pip install portalocker

In [2]:
import numpy as np
#PyTorch libraries
import torch
import torchtext
from torchtext.datasets import WikiText2
# Dataloader library
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
# Libraries to prepare the data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
# neural layers
from torch import nn
from torch.nn import functional as F
import torch.optim as optim
from tqdm import tqdm

import random

>> _device_ variable that we will use later to compute using the GPU if available, will use the CPU otherwise

In [3]:
# Use GPU if available
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
train_dataset, val_dataset, test_dataset = WikiText2()

In [ ]:
tokeniser = get_tokenizer('basic_english')
def yield_tokens(data):
    for text in data:
        yield tokeniser(text)

In [ ]:
# Build the vocabulary
vocab = build_vocab_from_iterator(yield_tokens(train_dataset), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
#set unknown token at position 0
vocab.set_default_index(vocab["<unk>"])

In [ ]:
seq_length = 50
def data_process(raw_text_iter, seq_length = 50):
    data = [torch.tensor(vocab(tokeniser(item)), dtype=torch.long) for item in raw_text_iter]
    data = torch.cat(tuple(filter(lambda t: t.numel() > 0, data))) #remove empty tensors
#     target_data = torch.cat(d)
    return (data[:-(data.size(0)%seq_length)].view(-1, seq_length),
            data[1:-(data.size(0)%seq_length-1)].view(-1, seq_length))

# # Create tensors for the training set
x_train, y_train = data_process(train_dataset, seq_length)
x_val, y_val = data_process(val_dataset, seq_length)
x_test, y_test = data_process(test_dataset, seq_length)

In [ ]:
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
test_dataset = TensorDataset(x_test, y_test)

In [ ]:
batch_size = 64  # choose a batch size that fits your computation resources
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [ ]:
# Define the LSTM model
# Feel free to experiment
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_size)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, text, hidden):
        embeddings = self.embeddings(text)
        output, hidden = self.lstm(embeddings, hidden)
        decoded = self.fc(output)
        return decoded, hidden

    def init_hidden(self, batch_size):

        return (torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device))



vocab_size = len(vocab) # vocabulary size
emb_size = 100 # embedding size
neurons = 128 # the dimension of the feedforward network model, i.e. # of neurons
num_layers = 1 # the number of nn.LSTM layers
model = LSTMModel(vocab_size, emb_size, neurons, num_layers)


In [ ]:

def train(model, train_loader, loss_function, optimizer, epochs):
    '''
    Train the LSTM model on the WikiText-2 dataset.

    Parameters:
    - model: The LSTM model to be trained.
    - train_loader: DataLoader for the training dataset.
    - loss_function: The loss function to be used during training.
    - optimizer: The optimizer to use for updating model parameters.
    - epochs: Number of epochs to train the model.

    The training process includes the following steps:
    - Loop through the specified number of epochs.
    - In each epoch, loop through the training data.
    - Zero the gradients before each batch to prevent accumulation.
    - Place data (input and target) on the device (GPU/CPU).
    - Initialize hidden states for the LSTM.
    - Run the model and compute the loss.
    - Perform backpropagation and update parameters.
    - Print epoch-wise loss and other relevant information.
    '''

    model = model.to(device=device)
    model.train()  # set the model to training mode

    for epoch in range(epochs):
        total_loss = 0

        for i, (data, targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")):
            # Place data on the correct device
            data, targets = data.to(device), targets.to(device)

            # Initialize hidden states
            hidden = model.init_hidden(data.size(0))

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass: compute predictions and loss
            output, _ = model(data, hidden)
            loss = loss_function(output.view(-1, vocab_size), targets.view(-1))

            # Backward pass: compute gradient and update parameters
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Average loss for the epoch
        average_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {average_loss:.4f}")


In [ ]:
# Call the train function
loss_function = nn.CrossEntropyLoss()
lr = 0.0005
epochs = 100
# Set up the optimizer and loss function for training
optimizer = optim.Adam(model.parameters(), lr=lr)
# Train the model
train(model, train_loader, loss_function, optimizer, epochs)


Epoch 1/100: 100%|██████████| 640/640 [00:07<00:00, 81.89it/s]


Epoch 1/100, Loss: 5.4538


Epoch 2/100: 100%|██████████| 640/640 [00:07<00:00, 84.85it/s]


Epoch 2/100, Loss: 5.3710


Epoch 3/100: 100%|██████████| 640/640 [00:07<00:00, 84.45it/s]


...

Epoch 98/100, Loss: 3.8355


Epoch 99/100: 100%|██████████| 640/640 [00:07<00:00, 83.23it/s]


Epoch 99/100, Loss: 3.8289


Epoch 100/100: 100%|██████████| 640/640 [00:07<00:00, 82.89it/s]

Epoch 100/100, Loss: 3.8225


In [ ]:
def generate_text(model, start_text, num_words, temperature=1.0):
    '''
    model.eval()
    words = tokeniser(start_text)
    hidden = model.init_hidden(1)
    for i in range(0, num_words):
        x = torch.tensor([[vocab[word] for word in words[i:]]], dtype=torch.long, device=device)
        y_pred, hidden = model(x, hidden)
        last_word_logits = y_pred[0][-1]
        p = (F.softmax(last_word_logits / temperature, dim=0).detach()).to(device='cpu').numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(vocab.lookup_token(word_index))

    return ' '.join(words)
    '''

    model.eval()  # Set the model to evaluation mode
    words = tokeniser(start_text)  # Tokenize the starting text
    hidden = model.init_hidden(1)  # Initialize hidden state

    # Generate text loop
    for i in range(num_words):
        x = torch.tensor([[vocab[word] for word in words[i:]]], dtype=torch.long, device=device)
        y_pred, hidden = model(x, hidden)
        last_word_logits = y_pred[0][-1]
        p = (F.softmax(last_word_logits / temperature, dim=0).detach()).to(device='cpu').numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(vocab.lookup_token(word_index))

    return ' '.join(words)

# Generate some text
print(generate_text(model, start_text="I like", num_words=100))


i like life . <unk> describes irresistible ' s cartoon spotlight . in the workforce and again eventually take the respectability to struggle . there is little note that the lack of appearance , such as the pilgrim ' s muybridge ' s head , as mariana ' s ? ' y ' t believe all @-@ faced crowds of the second week night , while doc gently violent outbursts and hits an added they did . another prosecutor that recur from the women ' s cold and three short games in 2010 during the 2003 season . a lap praised that
